### Take Picture

In [3]:
import pyrealsense2 as rs
import numpy as np
import cv2

i = 1
dst = 'RealSenseCamera/'

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:

        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        color_image = np.asanyarray(color_frame.get_data())

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', color_image)
        key = cv2.waitKey(100)
        if key == 27:
            break
        elif key == ord('c'):
            filename = 'pic{:0>3d}.jpg'.format(i)
            cv2.imwrite(dst + filename, color_image)
            print(filename + '已保存', end='\r')
            i += 1
finally:

    # Stop streaming
    cv2.destroyAllWindows()
    pipeline.stop()

### Locate Hand

In [ ]:
import cv2
import numpy as np

def locate_hand(input_img):
    img = input_img[:,:,0]
    height, width,  = img.shape[:2]
    background = img[0, 0]
    x_min, x_max, y_min = width, 0, height
    for i in range(height):
        for j in range(width):
            if img[i, j] != background:
                if x_min > j:
                    x_min = j
                if x_max < j:
                    x_max = j
                if y_min > i:
                    y_min = i
    return (x_min, y_min), (x_max, y_min + x_max - x_min)

### Remove Background

In [5]:
# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 0.5 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()
        # frames.get_depth_frame() is a 640x360 depth image
        
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)
        
        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()
        
        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue
        
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
          
        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 153
        depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), grey_color, color_image)
        bg_removed_depth = np.where((depth_image > clipping_distance) | (depth_image <= 0), 0, depth_image)
        
        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(bg_removed_depth, alpha=0.03), cv2.COLORMAP_JET)
        
        # ADD: locate hand using binarized image
        pt1, pt2 = locate_hand(depth_colormap)
        depth_colormap = cv2.rectangle(depth_colormap, pt1, pt2, (255, 255, 255))
        
        images = np.hstack((bg_removed, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.imwrite('depth.jpg', depth_colormap)
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()

Depth Scale is:  0.0010000000474974513
